# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [15]:
#Import the necessary libraries for EDA, visualization and time series modelling.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pm
%matplotlib inline

In [56]:
df = pd.read_csv('zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14723 entries, 0 to 14722
Columns: 272 entries, RegionID to 2018-04
dtypes: float64(219), int64(49), object(4)
memory usage: 30.6+ MB


In [132]:
florida = df[df['State']=='FL'].drop(['RegionID','City','State','Metro','CountyName','SizeRank'],axis=1)
florida.head()

,RegionName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
11,32162,101000.0,100700.0,100400.0,100100.0,99800.0,99500.0,99200.0,98900.0,98600.0,...,249900,250400,249900,248700,247500,247100,246800,247500,249800,251800
60,33160,337300.0,337900.0,338300.0,338400.0,338400.0,338200.0,338000.0,338000.0,338400.0,...,1466200,1471000,1485400,1497700,1504200,1507600,1495800,1466200,1443900,1432300
82,33025,111600.0,111700.0,111800.0,111900.0,112000.0,112100.0,112200.0,112300.0,112500.0,...,283500,286100,287800,288800,289700,291000,292500,294200,296400,298200
83,33411,126800.0,126800.0,126800.0,126800.0,126700.0,126700.0,126800.0,126800.0,127000.0,...,297400,299300,301400,303300,304900,307100,309800,311700,312600,313100
84,33139,480200.0,480700.0,481200.0,481600.0,482000.0,482500.0,483100.0,483700.0,484800.0,...,2917100,2925700,2942400,2958300,2946300,2912900,2900900,2923300,2933600,2914100


# Step 2: Data Preprocessing

In [134]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [135]:
dates = get_datetimes(florida)
dates

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [140]:
plt.style.available

['seaborn-dark',
 'seaborn-darkgrid',
 'seaborn-ticks',
 'fivethirtyeight',
 'seaborn-whitegrid',
 'classic',
 '_classic_test',
 'fast',
 'seaborn-talk',
 'seaborn-dark-palette',
 'seaborn-bright',
 'seaborn-pastel',
 'grayscale',
 'seaborn-notebook',
 'ggplot',
 'seaborn-colorblind',
 'seaborn-muted',
 'seaborn',
 'Solarize_Light2',
 'seaborn-paper',
 'bmh',
 'tableau-colorblind10',
 'seaborn-white',
 'dark_background',
 'seaborn-poster',
 'seaborn-deep']

# Step 3: EDA and Visualization

In [139]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

NameError: name 'matplotlib' is not defined

# Step 4: Reshape from Wide to Long Format

In [136]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [137]:
florida_melt = melt_data(florida).groupby(['time','RegionName']).mean()

In [138]:
florida_melt

value
time       RegionName          
1996-04-01 32003       135500.0
           32008        41400.0
           32009        74000.0
           32011        76700.0
           32024        53200.0
           32025        48000.0
           32034        97100.0
           32040        66400.0
           32043        87200.0
           32055        30600.0
           32060        49100.0
           32063        72600.0
           32064        38000.0
           32065       103800.0
           32068        82100.0
           32073        87700.0
           32080       126400.0
           32081       155800.0
           32082       219200.0
           32084        57400.0
           32086        84100.0
           32087        39500.0
           32092        99500.0
           32095       114800.0
           32097        74500.0
           32102        76700.0
           32112        50100.0
           32113        58600.0
           32114        47700.0
           32117        54000.0
...                         ...
2018-04-01 34758       178800.0
           34759       166100.0
           34761       244700.0
           34769       178900.0
           34771       294700.0
           34772       237300.0
           34773       241400.0
           34785        95500.0
           34786       429300.0
           34787       312900.0
           34788       162500.0
           34797       193500.0
           34945       216700.0
           34949       372900.0
           34951       154100.0
           34952       191000.0
           34953       215500.0
           34957       274000.0
           34972        99700.0
           34974       129800.0
           34981       213900.0
           34982       152500.0
           34983       190700.0
           34984       207200.0
           34986       247700.0
           34987       268100.0
           34990       377800.0
           34994       284300.0
           34996       506400.0
           34997       281700.0

[205265 rows x 1 columns]

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results